In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 5

# Create a DataFrame with image paths and labels
data = {'filename': image_paths, 'label': labels}
df = pd.DataFrame(data)

# Convert label values to string type
df['label'] = df['label'].astype(str)

# Split data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Model definition and training
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')  # 4 output classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_df) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=test_generator,
    validation_steps=len(test_df) // BATCH_SIZE
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy}")


Found 3414 validated image filenames belonging to 4 classes.
Found 854 validated image filenames belonging to 4 classes.
Epoch 1/5
106/106 [==============================] - 127s 1s/step - loss: 1.1669 - accuracy: 0.5651 - val_loss: 0.6486 - val_accuracy: 0.7680
Epoch 2/5
106/106 [==============================] - 126s 1s/step - loss: 0.5959 - accuracy: 0.7750 - val_loss: 0.5126 - val_accuracy: 0.8077
Epoch 3/5
106/106 [==============================] - 125s 1s/step - loss: 0.3915 - accuracy: 0.8548 - val_loss: 0.5035 - val_accuracy: 0.7969
Epoch 4/5
106/106 [==============================] - 131s 1s/step - loss: 0.3026 - accuracy: 0.8838 - val_loss: 0.2240 - val_accuracy: 0.9219
Epoch 5/5
27/27 [==============================] - 8s 304ms/step - loss: 0.2123 - accuracy: 0.9379
Test Accuracy: 0.9379391074180603


In [6]:
model.save("rice_disease_cnn.h5")

C:\Users\Dell\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Define constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 1

# Create a DataFrame with image paths and labels
data = {'filename': image_paths, 'label': labels}
df = pd.DataFrame(data)

# Convert label values to string type
df['label'] = df['label'].astype(str)

# Split data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='label',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Load the pre-trained VGG19 model (include_top=False to exclude the fully connected layers)
vgg_base = VGG19(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Freeze the layers in the pre-trained model
for layer in vgg_base.layers:
    layer.trainable = False

# Add custom classification layers on top of the VGG19 base
model = models.Sequential([
    vgg_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # 4 output classes
])

model.compile(optimizer=Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_df) // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=test_generator,
    validation_steps=len(test_df) // BATCH_SIZE
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy}")

# Save the VGG19-based model
model.save("rice_disease_vgg19.h5")


Found 3414 validated image filenames belonging to 4 classes.
Found 854 validated image filenames belonging to 4 classes.


27/27 [==============================] - 194s 7s/step - loss: 0.5079 - accuracy: 0.7939
Test Accuracy: 0.7939109802246094


C:\Users\Dell\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
